In [ ]:
from bs4 import BeautifulSoup 
import requests
import pandas as pd
import numpy as np
pd.set_option('display.max_colwidth', None)
import time

from tqdm.notebook import tqdm
import extruct
import pickle

In [ ]:
headers = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.5 Safari/605.1.15"
}

In [ ]:
sitemap_url = 'https://www.nytimes.com/sitemaps/new/cooking.xml.gz'
response = requests.get(sitemap_url, headers=headers)
sitemap_xml = response.text
sitemap_soup = BeautifulSoup(sitemap_xml, features='xml')

gz_urls = []
loc_tags = sitemap_soup.find_all('loc')

for loc in loc_tags:
    gz_urls.append(loc.get_text()) 

display(gz_urls)
print(len(gz_urls))

In [ ]:
with open('../data/nytc_raw_urls.pkl', 'rb') as f:
    recipe_urls = pickle.load(f)

In [ ]:
# recipe_urls = []

# for url in tqdm(gz_urls):
#     response = requests.get(url, headers=headers)
#     sitemap_soup = BeautifulSoup(response.text, features='xml')
#     loc_tags = sitemap_soup.find_all('loc')
#     for loc in loc_tags:
#         recipe_urls.append(loc.get_text()) 


In [ ]:
display(recipe_urls)
len(recipe_urls)

In [ ]:
# with open('../data/nytc_raw_urls.pkl', 'wb') as f:
#     pickle.dump(recipe_urls, f)

Quick and dirty prune?

In [ ]:
import random

random_url = random.choice(recipe_urls)
print(random_url)

In [ ]:
import extruct

page = requests.get(random_url, headers=headers)
recipe_soup = BeautifulSoup(page.text, 'html.parser')

recipe_data = extruct.extract(
            page.text,
            syntaxes=['json-ld'],
            uniform=True,
        )['json-ld'][0]
display(recipe_data)

In [ ]:
headline = recipe_data['name'] # or 'name'
nutrition = recipe_data['nutrition']
category = recipe_data['recipeCategory']
cuisine = recipe_data['recipeCuisine']
ingredient = recipe_data['recipeIngredient']
instruction = recipe_data['recipeInstructions'][0]['text']

print(headline)
print(nutrition)
print(category)
print(cuisine)
print(ingredient)
print(instruction)

Test out a few recipes first

In [ ]:
from nytc import NYTC

recipes = pd.Series(recipe_urls, name='url')
features = ['recipe_name', 'nutrition', 'category', 'cuisine', 'ingredient', 'instruction', 'raw_schema']

In [ ]:
feature_df = pd.DataFrame(columns=features)
for i in tqdm(range(10)):
    url = recipes.iloc[i]
    try:
        recipe = NYTC(url)
        feature_df.loc[i] = [getattr(recipe, feature)() for feature in features]
        time.sleep(np.random.uniform(0.05, 1))
    except:
        feature_df.loc[i] = [np.nan] * len(features)


In [ ]:
feature_df.head(3)

In [ ]:
feature_df = pd.DataFrame(columns=features)
for i in tqdm(range(len(recipe_urls))):
    url = recipes.iloc[i]
    try:
        recipe = NYTC(url)
        feature_df.loc[i] = [getattr(recipe, feature)() for feature in features]
        time.sleep(np.random.uniform(0.05, 1))
    except:
        feature_df.loc[i] = [np.nan] * len(features)

In [ ]:
data_df = pd.concat([recipes, feature_df], axis=1)

In [ ]:
data_df.to_pickle('../data/nytc_data.pkl')

Sanity Check

In [ ]:
data_df.sample(10)
